## Ebola infected Macaque Sample Composition Trajectory Identification

In this notebook we deconvolve the ebov macaque blood samples using a matched seq-well reference.

In [ ]:
import matplotlib.pylab as plt

In [ ]:
import torch
from ternadecov.time_deconv import *
from ternadecov.simulator import *
from ternadecov.stats_helpers import *

# Configuration

In [ ]:
device = torch.device("cpu:0")
dtype = torch.float32
dtype_np = np.float32
n_iters = 10_000

# Load data

In [ ]:
machine_name = 'remote'
    
if machine_name == 'remote':
    sc_anndata_path = "/home/nbarkas/disk2/deconvolution_method/datasets/ebov/02-EBOV_bulk/load_data_python/ebov_sc.h5ad"
    bulk_anndata_path = "/home/nbarkas/disk2/deconvolution_method/datasets/ebov/02-EBOV_bulk/load_data_python/ebov_bulk.h5ad"
if machine_name == 'local':
    bulk_anndata_path = "/home/nbarkas/disk1/work/deconvolution_method/datasets/ebov/load_data_python/ebov_bulk.h5ad"
    sc_anndata_path = "/home/nbarkas/disk1/work/deconvolution_method/datasets/ebov/load_data_python/ebov_sc.h5ad"

In [ ]:
with open(bulk_anndata_path, 'rb') as fh:
    bulk_anndata = anndata.read_h5ad(fh)
with open(sc_anndata_path, 'rb') as fh:
    sc_anndata = anndata.read_h5ad(fh)

In [ ]:
# select samples only after or on tp 0 
bulk_anndata = bulk_anndata[bulk_anndata.obs['dpi_time'] >= 0,]

In [ ]:
ebov_dataset = DeconvolutionDataset(
    sc_anndata = sc_anndata,
    sc_celltype_col = "Subclustering_reduced",
    bulk_anndata = bulk_anndata,
    bulk_time_col = "dpi_time",
    dtype_np = dtype_np,
    dtype = dtype,
    device=device,
    feature_selection_method = 'single_cell_od' #'overdispersed_bulk'
)

# Run Deconvolution

In [ ]:
pseudo_time_reg_deconv = TimeRegularizedDeconvolution(
    dataset=ebov_dataset,
    trajectory_model_type='gp',
    device=device,
    dtype=dtype)

In [ ]:
pseudo_time_reg_deconv.fit_model(n_iters=n_iters, verbose=True, log_frequency=1000, clear_param_store=False)

# Examine Outputs

In [ ]:
# Plot the loses
pseudo_time_reg_deconv.plot_loss()

In [ ]:
pyro.get_param_store().keys()

In [ ]:
pseudo_time_reg_deconv.plot_composition_trajectories()

In [ ]:
pseudo_time_reg_deconv.plot_composition_trajectories(show_sampled_trajectories = True, n_samples=500)

In [ ]:
pseudo_time_reg_deconv.plot_sample_compositions_scatter()

In [ ]:
# Do correlation between different cell types

In [ ]:
# This is now plotting a random sample
pseudo_time_reg_deconv.plot_sample_compositions_boxplot()
matplotlib.pyplot.show()

In [ ]:
pseudo_time_reg_deconv.plot_sample_compositions_boxplot()
matplotlib.pyplot.show()

In [ ]:
pseudo_time_reg_deconv.plot_sample_compositions_boxplot_confidence()